# 8.10- Reduccion de dimensiones

### PCA  (análisis de componente principal)

**PCA** es una transformación lineal usada para reducir dimensiones en los datos.

¿Por qué reducir dimensiones?

Existen varias razones, entre ellas:
+ Mejora de la performance
+ Mejor manejo de la dispersión de los datos
+ Maldición de las dimensiones (ojo-también existe la bendición)
+ etc...


Hay dos maneras diferentes de hacer ésta transformación:

+ A través de la matriz de correlaciones (dimensiones no homogéneas)
+ A través de la matriz de covarianzas (dimensiones homogéneas)

Ambas matrices son simétricas y diagonalizables. De hecho, el Teorema Espectral dice que si una matriz es hermítica, cuadrada y de dimensión finita, entonces existe una base de vectores propios donde dicha matriz puede ser representada.
Esto quiere decir que podemos cambiar de base para después proyectar, reduciendo las dimensiones e intentando conservar la máxima información en el nuevo subespacio.

![gio1](images/gioconda.jpeg)
![gio2](images/gioconda_lego.png)

#### Ejemplo intuitivo

In [ ]:
import pylab as plt
%matplotlib inline

In [ ]:
plt.figure(figsize=(10, 5))
plt.scatter([i for i in range(15)],
            [i+1 if i%2==0 else i-1 for i in range(15)])

plt.quiver(7, 7, 9, 4, color='r', scale=20)
plt.quiver(7, 7, -9, -4, color='r', scale=20)
plt.plot(7, 8, marker='$PC1$', ms=30, color='r')

plt.quiver(9, 9, -5, 4, color='b', scale=40)
plt.quiver(9, 9, 5, -4, color='b', scale=40)
plt.plot(8, 12, marker='$PC2$', ms=30, color='b');

Se rota y se proyecta, resultando:

In [ ]:
plt.figure(figsize=(10, 5))
plt.scatter([i for i in range(15)],
            [6 for i in range(15)])

plt.quiver(7, 6, 7, 0, color='r', scale=20)
plt.quiver(7, 6, -7, 0, color='r', scale=20)
plt.plot(7, 6.25, marker='$PC1$', ms=30, color='r');

**Combinacion lineal**

$v1>$ vector 1

$v2>$ vector 2

comb lineal = 2·$v1$ - 4·$v2$

**Combinacion no lineal**

comb no lineal = 2·$v1$·$v2$

##### Resumen PCA

+ Normalización de los datos
+ Obtener base de vectores propios desde matriz de correlacion o covarianza
+ Ordenar los vectores propios de mayor a menor según sus dimensiones en el nuevo subespacio
+ Matriz de proyección, con los autovectores seleccionados (W)
+ Se transforma X (los datos) según W (matriz de proyección)

In [ ]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler, MinMaxScaler # normalizar

from sklearn.decomposition import PCA

import warnings
warnings.simplefilter('ignore')

In [ ]:
data=pd.read_csv('../data/pulsar_stars.csv')

data.head()

In [ ]:
data=data.drop(columns=['target_class'])

#### normalización

Recordemos, el primer paso de PCA es la normalización de los datos. 

Primero, veamos la **estandarización**  ($N(\mu, \sigma)$):

$$\frac{x-\mu}{\frac{\sigma}{\sqrt{n}}}$$

In [ ]:
data_n_mano=(data - np.mean(data))/np.std(data)

data_n=StandardScaler().fit_transform(data)

np.sum(data_n_mano - data_n)

Ahora el **MinMax** :

$$\frac{x-min}{max-min}$$

In [ ]:
data_mm_mano=(data - np.min(data))/(np.max(data) - np.min(data))

data_mm=MinMaxScaler().fit_transform(data)

np.sum(data_mm_mano - data_mm)

Se usa la standarización:

In [ ]:
data=StandardScaler().fit_transform(data)

data=pd.DataFrame(data)

data.head()

In [ ]:
data.shape

Se aplica **PCA**

In [ ]:
pca=PCA()

pca.fit(data)

In [ ]:
pca.explained_variance_ratio_

In [ ]:
plt.figure(figsize=(10, 5))

plt.plot(np.cumsum(pca.explained_variance_ratio_))

plt.xlabel('Numero de componentes')
plt.ylabel('% varianza')
plt.ylim([0, 1.01]);

In [ ]:
pca=PCA(n_components=4)

df=pd.DataFrame(pca.fit_transform(data))

df.head()

In [ ]:
pd.DataFrame(data).head()  # datos originales

In [ ]:
pd.DataFrame(pca.inverse_transform(df)).head()   # proceso inverso de pca

### ICA (análisis de componente independiente)

Es la generalización de PCA. También es una transformación lineal, pero no requiere que los datos sigan una distribución Normal.


### Ejemplo
##### Problema de la fiesta:
Se intentan separar la voces de una grabación de audio...

In [ ]:
from scipy import signal
from sklearn.decomposition import FastICA

In [ ]:
time=np.linspace(0, 8, 2000)  # tiempo


s1=np.sin(2*time)  # señal seno

s2=np.sign(np.sin(3*time))  # señal onda cuadrada

s3=signal.sawtooth(2*np.pi*time)  # señal dientes de sierra

In [ ]:
S=np.c_[s1, s2, s3]  # concatenar señales

S+=0.1*np.random.normal(size=S.shape)  # ruido

S/=S.std(axis=0)  # normalizacion

In [ ]:
A=np.array([[1,1,1], [0.5, 2, 1], [1.5, 1, 2]])  # operador de mezcla de señal

X=np.dot(S, A.T)  # observaciones

In [ ]:
ica=FastICA(n_components=3)

S_ica=ica.fit_transform(X)  # señal extraida por ica

A_ica=ica.mixing_ # operador de mezcla señal 

In [ ]:
pca=PCA(n_components=3)  # pca para comparar

S_pca=pca.fit_transform(X)

In [ ]:
plt.figure(figsize=(10, 8))

nombres=['obser', 'verdad', 'ica', 'pca']

modelos=[X, S, S_ica, S_pca]

colores=['red', 'steelblue', 'yellow']

for i , (m,n) in enumerate(zip(modelos, nombres), 1):
    plt.subplot(4, 1, i)
    plt.title(n)
    
    for sig, c in zip(m.T, colores):
        plt.plot(sig, color=c)
        
plt.subplots_adjust(0.09, 0.04, 0.94, 0.94, 0.26, 0.46)

plt.show();

### UMAP (uniform manifold aprox and projection)

Tiene dos pasos:

+ KNN con pesos, según topología (grafo)
+ Se reduce la dimensión basándose en esa topología

https://umap-learn.readthedocs.io/en/latest/

In [ ]:
!pip install umap-learn

In [ ]:
import umap

import seaborn as sns

from sklearn.datasets import load_iris

In [ ]:
load_iris().data.shape

In [ ]:
load_iris().target.shape

In [ ]:
redu=umap.UMAP(n_components=2).fit_transform(load_iris().data)

redu.shape

In [ ]:
plt.scatter(redu[:, 0], redu[:,1], c=[sns.color_palette()[x] for x in load_iris().target])
plt.gca().set_aspect('equal', 'datalim')
plt.title('Proyeccion UMAP');

### t-SNE

**t-Distributed Stochastic Neighbor Embbeding**

Convierte similitudes entre los datos en probabilidad conjunta y trata de minimizar la divergencia _Kullback-Leibler_ (entropía relativa):

$$D_{KL}(P|Q)=\sum P(x)log(\frac{P(x)}{Q(x)})$$

https://scikit-learn.org/stable/auto_examples/manifold/plot_t_sne_perplexity.html

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
tsne=TSNE(n_components=2, perplexity=20)

emb=tsne.fit_transform(load_iris().data)

tsne_df=pd.DataFrame(emb, columns=['e1', 'e2'])

tsne_df.head()

In [ ]:
tsne_df.plot.scatter(x='e1', y='e2', c=[sns.color_palette()[x] for x in load_iris().target])
plt.title('Proyeccion t-SNE');

In [ ]:
tsne_df.shape